In [ ]:
!git clone https://github.com/hoyeoplee/MeLU.git

Cloning into 'MeLU'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Unpacking objects: 100% (39/39), done.
/content/MeLU


In [ ]:
%cd MeLU

/content/MeLU


## Preparing dataset

In [ ]:
import os
from data_generation import generate

In [ ]:
master_path= "./ml"

In [ ]:
if not os.path.exists("{}/".format(master_path)):
    os.mkdir("{}/".format(master_path))
    generate(master_path)

4832it [03:31, 22.81it/s]
1208it [00:54, 22.03it/s]
4769it [02:05, 37.86it/s]
1197it [00:35, 33.55it/s]


## Training a model

Our model needs support and query sets. The support set is for local update, and the query set is for global update.

In [ ]:
import torch
import pickle
from MeLU import MeLU
from options import config
from model_training import training

In [ ]:
config

{'batch_size': 32,
 'embedding_dim': 32,
 'first_fc_hidden_dim': 64,
 'inner': 1,
 'local_lr': 5e-06,
 'lr': 5e-05,
 'num_actor': 8030,
 'num_age': 7,
 'num_candidate': 20,
 'num_director': 2186,
 'num_epoch': 20,
 'num_gender': 2,
 'num_genre': 25,
 'num_occupation': 21,
 'num_rate': 6,
 'num_zipcode': 3402,
 'second_fc_hidden_dim': 64,
 'use_cuda': True}

In [ ]:
config['use_cuda'] = False
config['num_epoch'] = 1
config['embedding_dim'] = 8
config['first_fc_hidden_dim'] = 16

In [ ]:
melu = MeLU(config)
model_filename = "{}/models.pkl".format(master_path)
if not os.path.exists(model_filename):
    # Load training dataset.
    training_set_size = int(len(os.listdir("{}/warm_state".format(master_path))) / 4)
    supp_xs_s = []
    supp_ys_s = []
    query_xs_s = []
    query_ys_s = []
    for idx in range(training_set_size):
        supp_xs_s.append(pickle.load(open("{}/warm_state/supp_x_{}.pkl".format(master_path, idx), "rb")))
        supp_ys_s.append(pickle.load(open("{}/warm_state/supp_y_{}.pkl".format(master_path, idx), "rb")))
        query_xs_s.append(pickle.load(open("{}/warm_state/query_x_{}.pkl".format(master_path, idx), "rb")))
        query_ys_s.append(pickle.load(open("{}/warm_state/query_y_{}.pkl".format(master_path, idx), "rb")))
    total_dataset = list(zip(supp_xs_s, supp_ys_s, query_xs_s, query_ys_s))
    del(supp_xs_s, supp_ys_s, query_xs_s, query_ys_s)
    training(melu, total_dataset, batch_size=config['batch_size'], num_epoch=config['num_epoch'], model_save=True, model_filename=model_filename)
else:
    trained_state_dict = torch.load(model_filename)
    melu.load_state_dict(trained_state_dict)

RuntimeError: ignored

In [ ]:
!apt-get install tree

In [ ]:
!tree --du -h -C -L 2 .

.
├── [6.1K]  data_generation.py
├── [1.2K]  dataset.py
├── [3.0K]  embeddings.py
├── [2.8K]  evidence_candidate.py
├── [1.9K]  main.py
├── [5.8K]  MeLU.py
├── [308M]  ml
│   ├── [416K]  item_cold_state
│   ├── [4.0K]  log
│   ├── [304M]  m_movie_dict.pkl
│   ├── [2.0M]  m_user_dict.pkl
│   ├── [116K]  user_and_item_cold_state
│   ├── [100K]  user_cold_state
│   └── [364K]  warm_state
├── [1.0K]  model_training.py
├── [8.0K]  movielens
│   └── [4.0K]  ml-1m
├── [ 629]  options.py
├── [ 18K]  __pycache__
│   ├── [4.1K]  data_generation.cpython-37.pyc
│   ├── [1.3K]  dataset.cpython-37.pyc
│   ├── [2.3K]  embeddings.cpython-37.pyc
│   ├── [4.5K]  MeLU.cpython-37.pyc
│   ├── [ 983]  model_training.cpython-37.pyc
│   └── [ 590]  options.cpython-37.pyc
└── [4.1K]  README.md

 308M used in 9 directories, 17 files


## Extracting evidence candidates

We extract evidence candidate list based on the MeLU.

In [ ]:
from evidence_candidate import selection

evidence_candidate_list = selection(melu, master_path, config['num_candidate'])
for movie, score in evidence_candidate_list:
    print(movie, score)